## Multiplane PSFs to splines

In this example we'll create cubic splines from measured PSFS. This example requires that the `multiplane_measure_psf` notebook was previously run in the same directory as it uses the results from that notebook.

### Configuration

In [ ]:
import os
os.chdir("/home/hbabcock/Data/storm_analysis/jy_testing/")
print(os.getcwd())

if not os.path.exists("c1_psf.psf"):
    print("Please run the multiplane_measure_psf notebook first")

Create the multiplane analysis XML file that we'll need at the plane weighting step.

In [ ]:
import storm_analysis.jupyter_examples.multiplane_psfs_to_splines as multiplane_psfs_to_splines

multiplane_psfs_to_splines.multiplaneXML()

### PSF normalization

The first step is to normalize the PSFs relative to each other. 

The idea is that the relative PSF heights as a function of Z provides information about the localizations Z position. In order for this to work correctly we need the PSFs to be normalized.

In [ ]:
import storm_analysis.multi_plane.normalize_psfs as normalizePSFs

normalizePSFs.normalizePSFs(["c1_psf.psf", "c2_psf.psf"])

### Check Z offset

Make a plot of the PSFs heights versus Z for the purpose of visually checking that the z offset values are relatively well centered. If this turns out not to be the case then the z values "z_offset.txt" should be adjusted such that the PSFs are better centered.

Ideally, at least for standard biplane imaging, the maximum values of the two PSFs should be approximately symmetric around Z = 0.

In [ ]:
import storm_analysis.multi_plane.check_plane_offsets as checkPlaneOffsets

checkPlaneOffsets.checkPlaneOffsets(["c1_psf_normed.psf", "c2_psf_normed.psf"])


### Use normalized measured PSFs to create cubic splines

In [ ]:
import storm_analysis.spliner.psf_to_spline as psfToSpline

# 10 is 1/2 the spline size in pixels. This is reasonable value for setups with a camera pixel size of 100nm.
psfToSpline.psfToSpline("c1_psf_normed.psf", "c1_psf.spline", 10)
psfToSpline.psfToSpline("c2_psf_normed.psf", "c2_psf.spline", 10)


You can check the splines with a tool like ImageJ. Tiff versions of the splines are also created, "c1_psf_sp.tif" and "c2_psf_sp.tif" in this example.

### Creating the weights file

Multiplane uses channel “information” weights in order to more optimally weight the contribution from each plane in the determination of a localizations parameters. The channels are weighted based on their Cramer-Rao bounds as a function of z.

For this step you will need a reasonable estimate of the average background and intensity in the data you plan to analyze. The background is photo-electrons per plane. The signal is the expected average number of photo-electrons per localization summed over all the planes. If your camera does not have a gain of 1.0 you will need to convert camera counts to photo-electrons.

Ideally the analysis would update the Cramer-Rao bounds for each localization as the estimates for the background and intensity change. However as the Cramer-Rao calculation is somewhat computational expensive we forego this step and instead just use average values.

In [ ]:
import storm_analysis.multi_plane.plane_weighting as planeWeighting

planeWeighting.runPlaneWeighting("multiplane.xml", 
                                 "weights.npy",
                                 [20, 20],   # 20 photons average background in each plane
                                 2000,       # 2000 photon average PSF sum.
                                 no_plots = False)


In the above plots the colored lines are the Cramer-Rao bounds for each individual PSF and the black line is the Cramer-Rao bound for all of the PSFs together.